 <span style='background:gray;font-size:40px;'>  Setup  </span>

 <span style='background:gray;font-size:40px;'>  Importing Libraries  </span>

In [3]:

from google.cloud import storage

import os
import random
import json
import gzip
import csv
import itertools

import numpy as np
import pandas as pd

from urllib.request import urlopen
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict
#from amazon.api import AMAZONAPI

 <span style='background:gray;font-size:40px;'>  Jupyter Options  </span>

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_colwidth', 100)

In [ ]:
# Already running in my instance since I checkboxed Anaconda
#%pip install more-itertools

***

# Import from Bucket - (Not needed since i uploaded directly onto persistent SSD)

In [ ]:
bucket_name = "finalproject-storage"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# # When you have your files in a subfolder of the bucket.
# my_prefix = "/" # the name of the subfolder
# blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')

# for blob in blobs:
#     if(blob.name != my_prefix): # ignoring the subfolder itself 
#         file_name = blob.name.replace(my_prefix, "")
#         blob.download_to_filename(file_name) # download the file to the machine
#         df = pd.read_csv(file_name) # load the data
#         print(df)

# When you have your files on the first level of your bucket

blobs = bucket.list_blobs()


for blob in blobs:
    file_name = blob.name
    blob.download_to_filename(file_name) # download the file to the machine
    # df = pd.read_csv(file_name) # load the data
    # print(df)
    
# elements = bucket.list_blobs()
# files=[a.name for a in elements]
# files
# path=f"gs://finalproject-storage/notebooks/jupyter/xaa"

In [ ]:
# Instantiate a Google Cloud Storage client and specify required bucket and file
storage_client = storage.Client()
bucket = storage_client.get_bucket('alouis9')
blob = bucket.blob('Electronics_5.json')

# Download the contents of the blob as a string and then parse it using json.loads() method
#json_data = blob.download_as_string().decode('utf-8')


In [ ]:
bucket.list_blobs()
blob
bucket

In [ ]:
for blob in blobs:
    file_name = blob.name
    blob.download_to_filename(file_name)

In [ ]:
data = pd.read_csv("gs://alouis9/Electronics_5.json")

***

<span style='background:gray;font-size:40px;'>  View Local Persistent SSD Tmp Folder where datafiles are stored  </span>

In [1]:
import os
[s for s in os.listdir('/home/derek') if (s.endswith('.csv') | s.endswith('.gz'))]



['Musical_Instruments.json.gz',
 'meta_Electronics.csv',
 'edgelist.csv',
 'All_Amazon_Meta.json.gz',
 'Musical_Instruments_5.json.gz',
 'Electronics.json.gz',
 'Electronics_5.json.gz',
 'Musical_Instruments.csv',
 'nodeslist.csv',
 'meta_Electronics.json.gz',
 'title_alsobuy_asin_master.csv',
 'Electronics.csv']

# Nodes List w Image Link

In [ ]:
    path= '/home/derek/meta_Electronics.json.gz'
    def parse(path):
        g = gzip.open(path, 'rb')
        for l in g:
            yield json.loads(l)
    def getDF(whatev):
        i = 0
        df = {}
        for d in parse(whatev):
            df[i] = d
            i += 1
        return pd.DataFrame.from_dict(df, orient='index')

    dfnodes = getDF(path)
    dfnodes = dfnodes[['asin', 'title', 'also_buy','date', 'imageURLHighRes']].explode('imageURLHighRes')
    dfnodes = dfnodes[dfnodes.also_buy.str.len() > 0]   
    dfnodes = dfnodes.groupby('asin').first().reset_index()
    dfnodes['also_buys'] = dfnodes['also_buy'].agg(lambda x: ','.join(map(str, x)))
    dfnodes = dfnodes.sort_values(by='date', ascending=False).drop_duplicates(subset=['title','also_buys'], keep='first')
    dfnodes.to_csv(path.split('.')[0] + 'cleaned_' + '.csv', index=False)

In [72]:
dfnodes.head()
dfnodes[dfnodes.duplicated('asin')]
print("{}{:,}{}".format("There are ",dfnodes.asin.count()," unique asins w metadata \n"))


,asin,title,also_buy,date,imageURLHighRes,also_buys
14822,B0047AFZXG,Effy Pav= Classica Collection Sterling Silver .97 ct. T.W. Diamond Bangle,"[B0045EQCCW, B00ZSHDJ4O, B00MOUBYDQ, B00NEMG62M, B0033Z2BAQ, B01N0L41N7, B008X3JHJQ, B07BN5FJZQ,...",iStarUSA,https://images-na.ssl-images-amazon.com/images/I/415y-dsTZEL.jpg,"B0045EQCCW,B00ZSHDJ4O,B00MOUBYDQ,B00NEMG62M,B0033Z2BAQ,B01N0L41N7,B008X3JHJQ,B07BN5FJZQ,B07598VZ..."
57258,B00RV8KSIS,VOWSVOWS 2-Pack Premium Earphones Earbuds Headphones with Stereo MicRemote Control Compatible wi...,[B000RB0PUC],VOWSVOWS,https://images-na.ssl-images-amazon.com/images/I/41bB6ujxObL.jpg,B000RB0PUC
69512,B011N1N0TC,TOOGOO(R) 75 Leather Cord Ends Caps Necklace Clasp Jewelry Parts,"[B01NAFEMAF, B005VLG2N2, B009LIOFWQ, B008FL9X7C, B000OVJYSE, B00BBQCJZ6, B013MMQH90, B00WUC4HB4,...",TOOGOO(R),https://images-na.ssl-images-amazon.com/images/I/41%2BH3y4IBaL.jpg,"B01NAFEMAF,B005VLG2N2,B009LIOFWQ,B008FL9X7C,B000OVJYSE,B00BBQCJZ6,B013MMQH90,B00WUC4HB4,B01MQFCM..."
51229,B00NG0QJDI,"Samsung Gear S Screen Protector + Full Body, Skinomi TechSkin Full Coverage Skin + Screen Protec...","[B00PZDHAUA, B00SL1TI7Q, B00SM6J8XY, B01EPRJ9DC, B01G4H8T64, B01J4ZB492]",Skinomi,https://images-na.ssl-images-amazon.com/images/I/410JxHH3HoL.jpg,"B00PZDHAUA,B00SL1TI7Q,B00SM6J8XY,B01EPRJ9DC,B01G4H8T64,B01J4ZB492"
65053,B00XPINZ10,"USB 3.0 Cable Male to Male 6 ft, SNANSHI USB to USB Cable Nylon Braided Cable Aluminum Shell for...","[B00KOHQU58, B01LXR6DKV, B01IDIA0DW, B01LY97QE8, B01G3JNYEK]","September 9, 2018",https://images-na.ssl-images-amazon.com/images/I/41-eYBJqwSL.jpg,"B00KOHQU58,B01LXR6DKV,B01IDIA0DW,B01LY97QE8,B01G3JNYEK"


,asin,title,also_buy,date,imageURLHighRes,also_buys


There are 98,125 unique asins w metadata 




<span style='background:red;font-size:40px;'>  Import Ratings Only CSV Data  </span>
***

In [11]:
path = '/home/derek/Electronics.csv'
header_list = ["item", "user", "rating", "timestamp"]
types_dict = {'item': 'string', 'user': 'string', 'rating': float, 'timestamp':'string'}
df_ratings = pd.read_csv(path, names=header_list, dtype=types_dict)
print("{}{:,}{}".format("There are ",len(df_ratings)," reviews in this dataset \n"))
df_ratings.info()
df_ratings = df_ratings.merge(dfnodes,how="left", left_on=['item'], right_on=['asin'])[["item", "user", "rating", "timestamp", "title"]]
df_ratings.head()

There are 20,994,353 reviews in this dataset 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20994353 entries, 0 to 20994352
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   item       string 
 1   user       string 
 2   rating     float64
 3   timestamp  string 
dtypes: float64(1), string(3)
memory usage: 640.7 MB


,item,user,rating,timestamp,title
0,0060009810,A1N070NS9CJQ2I,5.0,1026864000,One Hot Summer
1,0060009810,A3P0KRKOBQK1KN,5.0,1025913600,One Hot Summer
2,0060009810,A192HO2ICJ75VU,5.0,1025654400,One Hot Summer
3,0060009810,A2T278FKFL3BLT,4.0,1025395200,One Hot Summer
4,0060009810,A2ZUXVTW8RXBXW,5.0,1025222400,One Hot Summer



<span style='background:green;font-size:40px;'> Metadata Exploration  </span>
***



<span style='background:green;font-size:40px;'> Rating Frequency Count </span>


In [9]:
#df_ratings.style.format('{:,}')
ratingsFreq = df_ratings.groupby('rating').count().sort_values(['rating'],ascending=False)
ratingsFreq.style.format('{:,}')
print("{:,}{}{:,}{}".format(ratingsFreq.item.iloc[0]," out of ", ratingsFreq.item.sum(), " total reviews were rated 5 stars \n"))

,item,user,timestamp,asin,title,imageURLHighRes
rating,,,,,,
5.0,"12,602,917","12,602,917","12,602,917","12,593,319","12,593,319","10,826,946"
4.0,"3,306,379","3,306,379","3,306,379","3,302,953","3,302,953","2,815,671"
3.0,"1,529,818","1,529,818","1,529,818","1,528,321","1,528,321","1,302,992"
2.0,"1,139,589","1,139,589","1,139,589","1,138,515","1,138,515","974,250"
1.0,"2,415,650","2,415,650","2,415,650","2,413,686","2,413,686","2,028,622"


12,602,917 out of 20,994,353 total reviews were rated 5 stars 



In [10]:
ratingspct = df_ratings.rating.value_counts(normalize=True)
print("{}{:,.1%}{}".format("",ratingspct.iloc[0]," of the reviews were rated 5 stars \n"))
ratingspct.to_frame().style.format('{:,.1%}')

60.0% of the reviews were rated 5 stars 



,rating
5.0,60.0%
4.0,15.7%
1.0,11.5%
3.0,7.3%
2.0,5.4%



<span style='background:green;font-size:40px;'> Average Rating Per Product </span>

In [11]:
df_ratings.groupby(['title','item'])['rating'].mean().sort_values(ascending=False)

title                                                                                                             item      
Incase Ebony/Clover Nylon Sling Sleeve for 17&quot; MacBook Pro                                                   B00GKRVJIE    5.0
Yonger 5 Pack Mini Multipurpose Cable Clips Organizer, Computer Electrical Charging or Mouse Cord Holder          B01DKGU4TM    5.0
Hot NEW White 4 Port USB 2.0 Mini Speed HUB Adapter man shape USB HUB                                             B00JUMIJ7A    5.0
Hot Movie The Hunger Games Design Hard Cover Cases Protective Skin For Ipod Touch 4 ipod4-NY133                   B00H72BIK0    5.0
Hot Lips Kissing Macbook Vinyl Decal Sticker (Red) By Boston Deals USA                                            B00TP2EPAA    5.0
                                                                                                                               ... 
General Electric Micro II Cassette Recorder                                        

<span style='background:green;font-size:40px;'> Top 25 Most Reviewed Products </span>

In [12]:
#num_reviews = df_ratings.groupby('item').count()
#num_reviews.sort_values(['rating'],ascending=False)
df_ratings.groupby(['title','item']).size().sort_values(ascending=False).head(25)#.agg('count')
#top100reviewers = rev_prolific.head(100).reset_index().copy()
#top100reviewers

title                                                                                                                                                                                        item      
OontZ Angle 3 Enhanced Stereo Edition IPX5 Splashproof Portable Bluetooth Speaker with Volume Booster AMP 10 Watts Power, Custom Bass Radiator, 100' Wireless Range Bluetooth 4.2            B010OYASRG    28539
NETGEAR N300 WiFi Range Extender (EX2700)                                                                                                                                                    B00L0YLRUW    20873
Cheetah Mounts APFMSB TV Wall Mount Bracket for 20-55&rdquo; TVs Up To VESA 400 and 115 lbs including a Twisted Veins 10&rsquo; HDMI Cable and a 6&quot; 3-Axis Magnetic Bubble Level        B00DIF2BO2    17045
Amazon Kindle 9W PowerFast Adapter for Accelerated Charging                                                                                                                  

<span style='background:green;font-size:40px;'> Top 10 Product with the most reviews per Rating </span>

In [13]:
#df_ratings.groupby(['rating', 'item']).size().sort_values(ascending=False).nlargest(10)
ratagg = df_ratings.groupby(['title','rating', 'item']).agg({'user':'nunique'}).reset_index()
ratsort = ratagg.groupby(['rating']).apply(lambda x: x.sort_values(['user'],ascending = False)).reset_index(drop = True)
ratsort.groupby(['rating']).head(10).sort_values(['rating', 'user'],ascending=False)


,title,rating,item,user
1169617,"OontZ Angle 3 Enhanced Stereo Edition IPX5 Splashproof Portable Bluetooth Speaker with Volume Booster AMP 10 Watts Power, Custom Bass Radiator, 100' Wireless Range Bluetooth 4.2",5.0,B010OYASRG,20411
1169618,"AmazonBasics High-Speed HDMI Cable - 9.8 Feet (3 Meters) Supports Ethernet, 3D, 4K and Audio Return",5.0,B003L1ZYYW,13269
1169619,"Mediabridge HDMI Cable (10 Feet) Supports 4K@60Hz, High Speed, Hand-Tested, HDMI 2.0 Ready - UHD, 18Gbps, Audio Return Channel",5.0,B0019HL8Q8,12577
1169620,NETGEAR N300 WiFi Range Extender (EX2700),5.0,B00L0YLRUW,12410
1169621,Cheetah Mounts APFMSB TV Wall Mount Bracket for 20-55&rdquo; TVs Up To VESA 400 and 115 lbs including a Twisted Veins 10&rsquo; HDMI Cable and a 6&quot; 3-Axis Magnetic Bubble Level,5.0,B00DIF2BO2,11659
1169622,"Fintie Slimshell Case for Kindle Paperwhite - Fits All Paperwhite Generations Prior to 2018 (Not Fit All-New Paperwhite 10th Gen), Black",5.0,B00ITIQI92,11565
1169623,"Mediabridge HDMI Cable (6 Feet) Supports 4K@60Hz, High Speed, Hand-Tested, HDMI 2.0 Ready - UHD, 18Gbps, Audio Return Channel",5.0,B0019EHU8G,11504
1169624,Amazon Kindle 9W PowerFast Adapter for Accelerated Charging,5.0,B006GWO5WK,10826
1169625,SanDisk 2GB Class 4 SD Flash Memory Card- SDSDB-002G-B35 (Label May Change),5.0,B000BQ7GW8,10718
1169626,Sabrent 4-Port USB 2.0 Hub with Individual Power Switches and LEDs (HB-UMLS),5.0,B00BWF5U0M,10279


<span style='background:green;font-size:40px;'> Top 25 Most Prolific Reviewers </span>

In [14]:
# rev_prolific = df_ratings.groupby('user').count().sort_values(['rating'],ascending=False)
# top100reviewers = rev_prolific.head(100).reset_index().copy()
# top100reviewers

df_ratings.groupby('user').size().sort_values(ascending=False).head(25)

user
A680RUE1FDO8B     633
A3OXHLG6DIBRW8    593
ADLVFFE4VBT8      549
A1X1CEGHTHMBL1    498
A6FIAB28IS79      491
A5JLAU2ARJ0BO     479
A31N0XY2UTB25C    471
A3OA4DV4L81N1D    424
A3LGT6UZL99IW1    424
A2LXX47A0KMJVX    418
A1EXGL6L0QQ0M5    411
A23GFTVIETX7DS    397
A1JZFGZEZVWQPY    384
A3AYSYSLHU26U9    376
A2XX2A4OJCDNLZ    375
A1XXMNLOLKNO0I    374
AWPODHOB4GFWL     372
A1T1YSCDW0PD25    370
ARBKYIVNYWK3C     370
AGVWTYW0ULXHT     364
A2D1LPEUCTNT8X    362
A22CW0ZHY3NJH8    359
ANGGNFE8DXGYC     357
A3CW0ZLUO5X2B1    355
A12DQZKRKTNF5E    350
dtype: int64

# Find The Product with most reviewer agreement for Each Rating

 <span style='background:red;font-size:40px;'> Load Category Product-Level Meta Data and create DF </span>

In [73]:
path = '/home/derek/meta_Electronics.json.gz'
metadata = []
with gzip.open(path) as f:
    for l in f:
        metadata.append(json.loads(l.strip()))

In [74]:
# convert list into pandas dataframe
df_metadata = pd.DataFrame.from_dict(metadata)
df_metadata['also_buys'] = df_metadata['also_buy'].agg(lambda x: ','.join(map(str, x)))
df_metadata = df_metadata.sort_values(by='date', ascending=False).drop_duplicates(subset=['title','also_buys'], keep='first')
print(len(df_metadata))
df_metadata.columns
len(df_metadata[df_metadata.isna().any(axis=1)]) # Rows with NANS
df_metadata.info()


744083


Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item',
       'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details',
       'also_buys'],
      dtype='object')

828

<class 'pandas.core.frame.DataFrame'>
Int64Index: 744083 entries, 546260 to 786444
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   category         744083 non-null  object
 1   tech1            744083 non-null  object
 2   description      744083 non-null  object
 3   fit              744083 non-null  object
 4   title            744083 non-null  object
 5   also_buy         744083 non-null  object
 6   tech2            744083 non-null  object
 7   brand            744083 non-null  object
 8   feature          744083 non-null  object
 9   rank             744083 non-null  object
 10  also_view        744083 non-null  object
 11  main_cat         744083 non-null  object
 12  similar_item     744083 non-null  object
 13  date             744083 non-null  object
 14  price            744083 non-null  object
 15  asin             744083 non-null  object
 16  imageURL         744083 non-null  object
 17  image

In [75]:
type(metadata)
len(metadata)
type(df_metadata)
len(df_metadata)

list

786445

pandas.core.frame.DataFrame

744083

In [76]:
print("{}{:,}{}".format("There are ",len(metadata)," products w metadata \n"))

# first row of the list
print(metadata[0])

There are 786,445 products w metadata 

{'category': ['Electronics', 'Camera &amp; Photo', 'Video Surveillance', 'Surveillance Systems', 'Surveillance DVR Kits'], 'tech1': '', 'description': ['The following camera brands and models have been tested for compatibility with GV-Software.\nGeoVision \tACTi \tArecont Vision \tAXIS \tBosch \tCanon\nCNB \tD-Link \tEtroVision \tHikVision \tHUNT \tIQEye\nJVC \tLG \tMOBOTIX \tPanasonic \tPelco \tSamsung\nSanyo \tSony \tUDP \tVerint \tVIVOTEK \t \n \nCompatible Standard and Protocol\nGV-System also allows for integration with all other IP video devices compatible with ONVIF(V2.0), PSIA (V1.1) standards, or RTSP protocol.\nONVIF \tPSIA \tRTSP \t  \t  \t \nNote: Specifications are subject to change without notice. Every effort has been made to ensure that the information on this Web site is accurate. No liability is assumed for incidental or consequential damages arising from the use of the information or products contained herein.'], 'fit': '', 'ti

 <span style='background:red;font-size:40px;'> Load MASTER Product-Level Meta Data (all categories) and create DF </span>

In [77]:
path = '/home/derek/title_alsobuy_asin_master.csv'
col_list = ["title", "asin"]
types_dict = {'title': 'string', 'alsobuy': 'string', 'asin': 'string'}
df_masterMETA = pd.read_csv(path, dtype=types_dict, usecols=col_list, skipinitialspace = True)

In [78]:
print("{}{:,}{}".format("There are ",len(df_masterMETA)," items in this dataset \n"))
df_masterMETA.info()
#df_masterMETA = df_ratings.merge(dfnodes,how="left", left_on=['item'], right_on=['asin'])#[["item", "user", "rating", "timestamp", "title"]]
df_masterMETA.head()

There are 15,023,059 items in this dataset 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15023059 entries, 0 to 15023058
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   title   string
 1   asin    string
dtypes: string(2)
memory usage: 229.2 MB


,title,asin
0,"Women Blouse, Ninasill Hooded Sweatshirt Coat Winter Warm Wool Zipper Pockets Cotton Coat Outwear",6305121869
1,Coolred-Women Retro Split Sexy Skinny Premium Qipao Maxi Dress Cheongsam,6318708057
2,Gaok Men's Retro Cotton Multi-Pocket Camo Cargo Shorts,6342506256
3,QIBOE Men's Baggy Jeans Denim Sweatpants Loose Pants,6342509379
4,Crazy Explosion-proof Lens Polarized Sunglasses Cycling Glasses Lenses,6342522081


<span style='background:red;font-size:40px;'>Build Subset df to eliminate asins with no alsobuys</span>

In [79]:
df_alsobuys = df_metadata[df_metadata.also_buy.str.len() > 0]
df_alsobuys[df_alsobuys.also_buy.isna()]

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details,also_buys


In [81]:
df_metadata = df_alsobuys

 <span style='background:green;font-size:40px;'>Duplicate ASINS in Metadata Table</span>

In [82]:
#df_metadata[df_metadata.duplicated('asin')].head()
len(df_metadata[df_metadata.duplicated('asin')])

0

<span style='background:green;font-size:40px;'> List of Duplicate asins in metadata</span>

In [83]:


asin_dupes = df_metadata[df_metadata.duplicated('asin')].sort_values(by=['asin']).asin.tolist()

len(asin_dupes)

asin_dupes[0:10]
print("{}{:,}{}".format("There are ",len(asin_dupes)," asins with more than one row \n"))


0

[]

There are 0 asins with more than one row 




<span style='background:green;font-size:40px;'>View Actual Duplicate Asins Side by Side in Metadata Table</span>

In [84]:


df_asindupes = pd.concat(g for _, g in df_metadata.groupby('asin') if len(g) > 1)
df_asindupes.shape
print("{}{:,}{}{:,}{}".format("Since there are ",len(asin_dupes)," asins with more than one row, there are ", len(df_asindupes)," rows in total \n"))
df_asindupes.head(10)


ValueError: No objects to concatenate

# These are the rows with NAN in Details Column

In [ ]:
#df_metadata[df_metadata.isna().any(axis=1)]

In [85]:
df_metadata[df_metadata.asin=='B01NAJ3KQB']

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details,also_buys


 <span style='background:red;font-size:40px;'> ***FILTER METADATA DF***</span>

In [86]:
df_metadata[df_metadata.asin=='B00JHKSMG8']

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details,also_buys


***

 <span style='background:orange;font-size:40px;'>Co-purchases Search</span>

In [87]:
no_copurch = len(df_metadata[(df_metadata.also_buy.str.len() == 0) & (df_metadata.also_view.str.len() == 0)])
both_copurch = len(df_metadata[(df_metadata.also_buy.str.len() > 0) & (df_metadata.also_view.str.len() > 0)])

print("{}{:,}{}".format("There are ",len(df_metadata)," asins in total in the metadata table \n"))

print("{}{:,}{}".format("There are ",no_copurch," asins with no also_buys or also_views \n"))

print("{}{:,}{}".format("There are ",len(df_metadata[df_metadata.also_buy.str.len() == 0])," asins with no also_buys \n"))

print("{}{:,}{}".format("There are ",len(df_metadata[df_metadata.also_view.str.len() == 0])," asins with no also_views \n"))

print("{}{:,}{}".format("There are ",len(df_metadata[df_metadata.also_buy.str.len() > 0])," asins with at least one also_buy co-purchase \n"))

print("{}{:,}{}".format("There are ",len(df_metadata[df_metadata.also_view.str.len() > 0])," asins with at least one also_views co-purchase \n"))

print("{}{:,}{}".format("There are ",both_copurch," asins with at least one of both co-purchase types \n"))

df_metadata[(df_metadata.also_buy.str.len() > 0) & (df_metadata.also_view.str.len() > 0)].head()
df_graph = df_metadata[(df_metadata.also_buy.str.len() > 0) & (df_metadata.also_view.str.len() > 0)]


There are 98,125 asins in total in the metadata table 

There are 0 asins with no also_buys or also_views 

There are 0 asins with no also_buys 

There are 54,764 asins with no also_views 

There are 98,125 asins with at least one also_buy co-purchase 

There are 43,361 asins with at least one also_views co-purchase 

There are 43,361 asins with at least one of both co-purchase types 



,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details,also_buys
203689,"[Electronics, Computers & Accessories, Computer Components, External Components, Computer Cases,...",,[Update your jewelry collection with the diamond studded design of this Pave Classica bangle bra...,,Effy Pav= Classica Collection Sterling Silver .97 ct. T.W. Diamond Bangle,"[B0045EQCCW, B00ZSHDJ4O, B00MOUBYDQ, B00NEMG62M, B0033Z2BAQ, B01N0L41N7, B008X3JHJQ, B07BN5FJZQ,...",,Effy,"[Case Type: 4U Rackmount, Material: Front Bezel- Plastic, Handle- SECC Zinc-Coated Steel, Main C...",[],"[B0055EV30W, B0091IZ1ZG, B0056OUTBK, B07957B2VF, B00A7NBO6E, B00N9CXGSO, B003U03GP8, B01F7RJPHO,...",All Electronics,"class=""a-bordered a-horizontal-stripes a-spacing-extra-large a-size-base comparison_table"">\n\...",iStarUSA,$3.80,B0047AFZXG,"[https://images-na.ssl-images-amazon.com/images/I/415y-dsTZEL._US40_.jpg, https://images-na.ssl-...","[https://images-na.ssl-images-amazon.com/images/I/415y-dsTZEL.jpg, https://images-na.ssl-images-...",{},"B0045EQCCW,B00ZSHDJ4O,B00MOUBYDQ,B00NEMG62M,B0033Z2BAQ,B01N0L41N7,B008X3JHJQ,B07BN5FJZQ,B07598VZ..."
583739,"[Electronics, Headphones, Earbud Headphones, High performance sound quality with super dynamic d...",,"[, This headphones combining classic design, noise reduction, give you premium sound quality and...",,VOWSVOWS 2-Pack Premium Earphones Earbuds Headphones with Stereo MicRemote Control Compatible wi...,[B000RB0PUC],,VOWSVOWS,[High performance sound quality with super dynamic deep bass high resolution treble & crisp powe...,"285,502 in Cell Phones & Accessories (","[B07H42R87Q, B00EJDD2ZY, B075M81LSQ, B00OH1GI0E, B008U515LU, B073DX8XP8, B07H5J62MT, B000OVPFIW,...","<img src=""https://images-na.ssl-images-amazon.com/images/G/01/nav2/images/gui/amazon-fashion-sto...",,VOWSVOWS,,B00RV8KSIS,"[https://images-na.ssl-images-amazon.com/images/I/41bB6ujxObL._US40_.jpg, https://images-na.ssl-...","[https://images-na.ssl-images-amazon.com/images/I/41bB6ujxObL.jpg, https://images-na.ssl-images-...",{},B000RB0PUC
540993,"[Electronics, Wearable Technology, The Skinomi TechSkin Samsung Gear S full body protector is sp...",,"[<b>Protect your Investment</b>, What if there was a way to keep your mobile device looking as p...",,"Samsung Gear S Screen Protector + Full Body, Skinomi TechSkin Full Coverage Skin + Screen Protec...","[B00PZDHAUA, B00SL1TI7Q, B00SM6J8XY, B01EPRJ9DC, B01G4H8T64, B01J4ZB492]",,Skinomi,[The Skinomi TechSkin Samsung Gear S full body protector is specifically designed to provide com...,"1,043,501 in Clothing, Shoes & Jewelry (",[B00PZDHAUA],"<img src=""https://images-na.ssl-images-amazon.com/images/G/01/nav2/images/gui/amazon-fashion-sto...",,Skinomi,$12.95,B00NG0QJDI,"[https://images-na.ssl-images-amazon.com/images/I/410JxHH3HoL._SR38,50_.jpg, https://images-na.s...","[https://images-na.ssl-images-amazon.com/images/I/410JxHH3HoL.jpg, https://images-na.ssl-images-...",{},"B00PZDHAUA,B00SL1TI7Q,B00SM6J8XY,B01EPRJ9DC,B01G4H8T64,B01J4ZB492"
630229,"[Electronics, Computers & Accessories, Computer Accessories & Peripherals, Cables & Interconnect...",,[],,"USB 3.0 Cable Male to Male 6 ft, SNANSHI USB to USB Cable Nylon Braided Cable Aluminum Shell for...","[B00KOHQU58, B01LXR6DKV, B01IDIA0DW, B01LY97QE8, B01G3JNYEK]",,SNANSHI,"[Compatible with Laptop and Windows PC desktop; Laptop Cooling Pad, Car Bluetooth Speaker, DVD p...","[>#8,094 in Industrial & Scientific > Industrial Electrical > Wiring & Connecting > Cables & Int...","[B01M719D7Q, B07B8TKYJY, B07DNMKVSL, B00HSS9KJY, B075CZHY1N, B01G3JV5UA, B00HSS9JMW, B00P0E3954,...",Computers,"class=""a-bordered a-horizontal-stripes a-spacing-extra-large a-size-base comparison_table"">\n\...","September 9, 2018",$7.99,B00XPINZ10,"[https://images-na.ssl-images-amazon.com/images/I/41-eYBJqwSL._SS40_.jpg, https://images-na.ssl-...","[https://images-na.ssl-images-amazon.com/images/I/41-eYB

<span style='background:orange;font-size:40px;'>Alternative Methodology to find Alsobuys exist?</span>

In [88]:

df_metadata["also_buy"].apply(lambda x: 1 if len(x) == 0 else 0).sum()

0

<span style='background:orange;font-size:40px;'>Build Subset df to eliminate asins with no alsobuys</span>

In [89]:
df_alsobuys = df_metadata[df_metadata.also_buy.str.len() > 0]

In [90]:
len(df_alsobuys)
print("{}{:,}{}".format("There are ",len(df_metadata[df_metadata.also_buy.str.len() > 0])," asins with at least one also_buy co-purchase \n"))

98125

There are 98,125 asins with at least one also_buy co-purchase 



<span style='background:orange;font-size:70px;'>At this point there is no reason to keep working with the full category metadata set, so we can eliminate any rows with ZERO also-buys</span>

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Turn this flag on to replace full metadata df with subset limited to asins with alsobuys</b></p>
</div>

In [91]:
#df_metadata = df_alsobuys
df_metadata['asin'].str.strip()
len(df_metadata)
len(df_metadata)==len(df_alsobuys)

203689    B0047AFZXG
583739    B00RV8KSIS
653462    B011N1N0TC
540993    B00NG0QJDI
630229    B00XPINZ10
             ...    
286651    B007ABANFY
286750    B007AJZF4U
286762    B007AJZN1K
286759    B007AJZMIO
786444    B01HJH42KU
Name: asin, Length: 98125, dtype: object

98125

True

In [92]:
df_metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details,also_buys
203689,"[Electronics, Computers & Accessories, Computer Components, External Components, Computer Cases,...",,[Update your jewelry collection with the diamond studded design of this Pave Classica bangle bra...,,Effy Pav= Classica Collection Sterling Silver .97 ct. T.W. Diamond Bangle,"[B0045EQCCW, B00ZSHDJ4O, B00MOUBYDQ, B00NEMG62M, B0033Z2BAQ, B01N0L41N7, B008X3JHJQ, B07BN5FJZQ,...",,Effy,"[Case Type: 4U Rackmount, Material: Front Bezel- Plastic, Handle- SECC Zinc-Coated Steel, Main C...",[],"[B0055EV30W, B0091IZ1ZG, B0056OUTBK, B07957B2VF, B00A7NBO6E, B00N9CXGSO, B003U03GP8, B01F7RJPHO,...",All Electronics,"class=""a-bordered a-horizontal-stripes a-spacing-extra-large a-size-base comparison_table"">\n\...",iStarUSA,$3.80,B0047AFZXG,"[https://images-na.ssl-images-amazon.com/images/I/415y-dsTZEL._US40_.jpg, https://images-na.ssl-...","[https://images-na.ssl-images-amazon.com/images/I/415y-dsTZEL.jpg, https://images-na.ssl-images-...",{},"B0045EQCCW,B00ZSHDJ4O,B00MOUBYDQ,B00NEMG62M,B0033Z2BAQ,B01N0L41N7,B008X3JHJQ,B07BN5FJZQ,B07598VZ..."
583739,"[Electronics, Headphones, Earbud Headphones, High performance sound quality with super dynamic d...",,"[, This headphones combining classic design, noise reduction, give you premium sound quality and...",,VOWSVOWS 2-Pack Premium Earphones Earbuds Headphones with Stereo MicRemote Control Compatible wi...,[B000RB0PUC],,VOWSVOWS,[High performance sound quality with super dynamic deep bass high resolution treble & crisp powe...,"285,502 in Cell Phones & Accessories (","[B07H42R87Q, B00EJDD2ZY, B075M81LSQ, B00OH1GI0E, B008U515LU, B073DX8XP8, B07H5J62MT, B000OVPFIW,...","<img src=""https://images-na.ssl-images-amazon.com/images/G/01/nav2/images/gui/amazon-fashion-sto...",,VOWSVOWS,,B00RV8KSIS,"[https://images-na.ssl-images-amazon.com/images/I/41bB6ujxObL._US40_.jpg, https://images-na.ssl-...","[https://images-na.ssl-images-amazon.com/images/I/41bB6ujxObL.jpg, https://images-na.ssl-images-...",{},B000RB0PUC
653462,"[Electronics, Accessories & Supplies, Audio & Video Accessories, This is a new set of 75 nickel ...",,[<b>* TOOGOO is a registered trademark. ONLY Authorized seller of TOOGOO can sell under TOOGOO l...,,TOOGOO(R) 75 Leather Cord Ends Caps Necklace Clasp Jewelry Parts,"[B01NAFEMAF, B005VLG2N2, B009LIOFWQ, B008FL9X7C, B000OVJYSE, B00BBQCJZ6, B013MMQH90, B00WUC4HB4,...",,TOOGOO(R),"[This is a new set of 75 nickel plated coil cord ends, Great for making necklaces and bracelets,...","3,159,074 in Clothing, Shoes & Jewelry (",[],"<img src=""https://images-na.ssl-images-amazon.com/images/G/01/nav2/images/gui/amazon-fashion-sto...",,TOOGOO(R),$3.35,B011N1N0TC,"[https://images-na.ssl-images-amazon.com/images/I/41%2BH3y4IBaL._US40_.jpg, https://images-na.ss...","[https://images-na.ssl-images-amazon.com/images/I/41%2BH3y4IBaL.jpg, https://images-na.ssl-image...",{},"B01NAFEMAF,B005VLG2N2,B009LIOFWQ,B008FL9X7C,B000OVJYSE,B00BBQCJZ6,B013MMQH90,B00WUC4HB4,B01MQFCM..."
540993,"[Electronics, Wearable Technology, The Skinomi TechSkin Samsung Gear S full body protector is sp...",,"[<b>Protect your Investment</b>, What if there was a way to keep your mobile device looking as p...",,"Samsung Gear S Screen Protector + Full Body, Skinomi TechSkin Full Coverage Skin + Screen Protec...","[B00PZDHAUA, B00SL1TI7Q, B00SM6J8XY, B01EPRJ9DC, B01G4H8T64, B01J4ZB492]",,Skinomi,[The Skinomi TechSkin Samsung Gear S full body protector is specifically designed to provide com...,"1,043,501 in Clothing, Shoes & Jewelry (",[B00PZDHAUA],"<img src=""https://images-na.ssl-images-amazon.com/images/G/01/nav2/images/gui/amazon-fashion-sto...",,Skinomi,$12.95,B00NG0QJDI,"[https://images-na.ssl-images-amazon.com/images/I/410JxHH3HoL._SR38,50_.jpg, https://images-na.s...","[https://images-na.ssl-images-amazon.com/images/I/410JxHH3HoL.jpg, https://images-na.ssl-images-.

***

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">There are multiple ways to build an edgelist</b></p>
</div>

# Create Dictionary Form of Co-Purchases by Asin

In [93]:
# There are two different dictionary structures


## Dictionary where the keys are ['asin','title', 'also_buy'] and the values are a dictionary with indexed, attrb

In [94]:
alsodict = df_metadata[['asin','title', 'also_buy']].to_dict()


In [95]:
# [{0: '0011300000',
#   1: '0043396828',
#   2: '0060009810',

#list(alsodict.values())[:1]

# [{0: 'Genuine Geovision 1 Channel 3rd Party NVR IP Software with USB Dongle Onvif PSIA',
#   1: 'Books "Handbook of Astronomical Image Processing" with CD ROM, 2nd Edition, Hardcover Book by Berry &amp; Burnell',
#   2: 'One Hot Summer',
#   3: 'Hurray for Hattie Rabbit: Story and pictures (An Early I can read book)',
#   4: 'sex.lies.murder.fame.: A Novel',
#   5: 'College Physics',

In [96]:
## Dict where the ASIN is the key and 

In [97]:
#metadf[['asin','title', 'also_buy']].dropindex()
meta_df = df_metadata[['asin','title', 'also_buy']].set_index('asin')
#meta_dffinal = df_graph[['asin','title', 'also_buy']].set_index('asin')
meta_dict = meta_df.to_dict()
#meta_dictfinal = meta_dffinal.to_dict()


## Dict where the keys are ['title', 'also_buy'] and the values are [asin, attr]

In [98]:
meta_df.head()
#list(meta_dict.items())[0:1]
meta_dict.keys()

,title,also_buy
asin,,
B0047AFZXG,Effy Pav= Classica Collection Sterling Silver .97 ct. T.W. Diamond Bangle,"[B0045EQCCW, B00ZSHDJ4O, B00MOUBYDQ, B00NEMG62M, B0033Z2BAQ, B01N0L41N7, B008X3JHJQ, B07BN5FJZQ,..."
B00RV8KSIS,VOWSVOWS 2-Pack Premium Earphones Earbuds Headphones with Stereo MicRemote Control Compatible wi...,[B000RB0PUC]
B011N1N0TC,TOOGOO(R) 75 Leather Cord Ends Caps Necklace Clasp Jewelry Parts,"[B01NAFEMAF, B005VLG2N2, B009LIOFWQ, B008FL9X7C, B000OVJYSE, B00BBQCJZ6, B013MMQH90, B00WUC4HB4,..."
B00NG0QJDI,"Samsung Gear S Screen Protector + Full Body, Skinomi TechSkin Full Coverage Skin + Screen Protec...","[B00PZDHAUA, B00SL1TI7Q, B00SM6J8XY, B01EPRJ9DC, B01G4H8T64, B01J4ZB492]"
B00XPINZ10,"USB 3.0 Cable Male to Male 6 ft, SNANSHI USB to USB Cable Nylon Braided Cable Aluminum Shell for...","[B00KOHQU58, B01LXR6DKV, B01IDIA0DW, B01LY97QE8, B01G3JNYEK]"


dict_keys(['title', 'also_buy'])

In [99]:
# [('title',
#   {'0011300000': 'Genuine Geovision 1 Channel 3rd Party NVR IP Software with USB Dongle Onvif PSIA',
#    '0043396828': 'Books "Handbook of Astronomical Image Processing" with CD ROM, 2nd Edition, Hardcover Book by Berry &amp; Burnell',
#    '0060009810': 'One Hot Summer',
#    '0060219602': 'Hurray for Hattie Rabbit: Story and pictures (An Early I can read book)',

In [100]:
alsodict.keys()
meta_dict.keys()
#meta_dictfinal.keys()
type(meta_dict['also_buy'])
#len(meta_dictfinal['also_view'])
#meta_df
#meta_dict['also_buy']

dict_keys(['asin', 'title', 'also_buy'])

dict_keys(['title', 'also_buy'])

dict

# ASINS Sorted/Ranked by Number of Also_Buy

In [101]:
def sort_by_values_len(dict):
    dict_len= {key: len(value) for key, value in dict.items()}
    import operator
    sorted_key_list = sorted(dict_len.items(), key=operator.itemgetter(1), reverse=True)
    sorted_dict = [{item[0]: dict[item [0]]} for item in sorted_key_list]
    return sorted_dict

# Sorted Dictionary where key is asin and value is alsobuys list

In [102]:
alsobuys_rnk = sort_by_values_len(meta_dict['also_buy'])

In [103]:
#sorted(meta_dict['also_buy'].items(), key=lambda e: len(e))
type(alsobuys_rnk)
len(alsobuys_rnk)

type(alsobuys_rnk[0])
len(alsobuys_rnk[0:2])

list

98125

dict

2

In [104]:

# This Dictionary is 2996 rows less which is correct because it has removed the duplicate rows and combined the values

## Verification that the dictionary is ranked and the 100,776th row has no alsobuys (therefore doesn't exist)

In [110]:
# You can see how the dictionary is ranked by the number of alsobuys
alsobuys_rnk[1].values()
alsobuys_rnk[len(alsobuys_rnk)-1].values()
alsobuys_rnk[len(alsobuys_rnk)].values()


dict_values([['B077Y62GSJ', 'B018XX8K66', 'B01717TLMM', 'B01FWAZEIU', 'B015PRO512', 'B00IIWC2W6', 'B06VY6FXMM', 'B017T2QB22', 'B077D54W58', 'B013SKIC7G', 'B012AQIOXQ', 'B00JZVUBFE', 'B0713XMHH9', 'B01KANYY8U', 'B01JJMD29U', 'B01C9KYUG8', 'B0784SLQM6', 'B003SALVN4', 'B06Y45V625', 'B015PRCBBI', 'B01N9FIELY', 'B005LDLUW8', 'B003TWO8JA', 'B06X17CLQX', 'B01MU3WUX1', 'B079DSW6XX', 'B01FGCJE26', 'B07FB1CWXW', 'B00Z08TZNU', 'B01N1VMBUR', 'B07DL6X6ZJ', 'B00UZGDWKY', 'B0000BVYT3', 'B0095EFXMC', 'B07CKG5LJF', 'B00CPPXKPK', 'B00BGTPPSQ', 'B01HDC21FE', 'B001PS9E5I', 'B00077INZU', 'B005JN9310', 'B01FGCJE3A', 'B0163HP38W', 'B078D6KZ98', 'B0183FSG68', 'B00LPK5W66', 'B00SW9JI9A', 'B073SBZ8YH', 'B01609AF22', 'B00MPVR50A', 'B0000TO0BQ', 'B008JJLZ7G', 'B016XYQWXU', 'B079HR85C3', 'B07864WMK8', 'B00GG1ACX2', 'B013SKIBL8', 'B00BGTPQO4', 'B075YTHP95', 'B0781Z7Y3S', 'B006YG8X9Y', 'B01M4NANZ6', 'B00Z05JMKO', 'B01FGCJAHU', 'B00T5WVR24', 'B001D1UTC4', 'B01FHBZF7E', 'B0042G49XM', 'B008X3JHJQ', 'B00006B7DA', 'B0779

dict_values([['B00Z7SAZXC']])

IndexError: list index out of range

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Create an Edge Set Manually</b></p>
</div>

In [111]:

edgeset = set()
for i in range(len(alsobuys_rnk)):
    for asin in alsobuys_rnk[i]: 
        for alsobuy in alsobuys_rnk[i][asin]: # For Every Alsobuy in an alsobuy list
            edgeset.add(tuple(sorted((asin,alsobuy)))) # changes order of tuples 
        


<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Create an Edge LIST</b></p>
</div>

In [112]:
edgelist = []

for i in range(len(alsobuys_rnk)):
    for asin in alsobuys_rnk[i]:
        for alsobuy in alsobuys_rnk[i][asin]:
            edgelist.append((asin, alsobuy))


<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Note that they are of Different Lengths (probably list has duplicates)</b></p>
</div>

In [113]:
type(edgeset)
len(edgeset)
type(edgelist)
len(edgelist)
# The edgelist has 61,000 more than the set

set

1774016

list

1835272

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">When Creating a Set from the List this way, the number of edges is the same</b></p>
</div>


In [114]:
res = set(map(lambda x: tuple(sorted(x)), edgelist))

In [115]:
len(res) #Interesting same length as set above
res==edgeset

1774016

True

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">The above is proof that creating the set of edges is accurate and the list is not</b></p>
</div>


In [116]:
# Same number of asins
len(meta_dict['also_buy'].keys())
len(alsobuys_rnk)

98125

98125

***

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Annabels formula was built using the raw metadata file, not the df, so to make is less expensive I am lobbing off all rows we already know have no alsobuys<br>
<br>I already turned on the flag above to make this conversion so I am just reverting the df back to a list
<br>Create SUBSET METADATA TO LOWER COMPUTATION COST OF ANNABELS FORMULA</b></p>
</div>

In [117]:
metadatab = df_metadata.to_dict('records')

In [118]:
type(metadatab)
len(metadatab)

list

98125



<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Correct Edgelist Approach w Weights</b></p>
</div>


In [119]:
dict2 = {}

for item1 in metadatab: 
    for item2 in item1['also_buy']:
        if (item1['asin'],item2) and (item2, item1['asin']) not in dict2: 
            dict2[(item1['asin'],item2)] = 1
        elif (item1['asin'], item2) in dict2: 
            dict2[(item1['asin'],item2)] += 1
        elif (item2, item1['asin']) in dict2:
            dict2[(item2,item1['asin'])] += 1
    list_of_pairs = [(item1['also_buy'][p1], item1['also_buy'][p2]) for p1 in range(len(item1['also_buy'])) for p2 in range(p1+1,len(item1['also_buy']))]
    for (item3,item4) in list_of_pairs: 
#          if(item3, item4) and (item4, item3) not in dict1: 
#              dict1[(item3,item4)] = 1
        if (item3,item4) in dict2: 
            dict2[(item3,item4)] += 1
            break
        elif (item4,item3) in dict2: 
            dict2[(item4,item3)] += 1

In [120]:
# In this formula we take all the pairs of items in a single also bought list and then check to see if they are already in the edgelist. If they are we increase their weight by 1
# This is correct because we are not adding any new edges we are just giving freqently seen alsobought added focus/weight

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Basic Unweighted Edgeweight Approach, identical to networkx and my approach above</b></p>
</div>


In [121]:
#len(dict1)
len(dict2)


#max(dict1.values())
max(dict2.values())





#dict(itertools.islice(dict2.items(), 10))
#sorted(dict2.items(), key=lambda x: x[1],reverse=True)

# 29319645
# 1818275
# 1818275
# 271
# 109
# 3

1774016

223


<div class="alert alert-block alert-info">
    <b><p style="font-size:30px"> Create Edge set using the networkx built-in function</b></p>
</div>

In [122]:
import networkx as nx
g = nx.from_dict_of_lists(meta_dict['also_buy'])


In [123]:
len(meta_dict['also_buy'])

98125

# It has the same number of edges as our manual edgeliset but different number of nodes?

In [124]:
g.size()
type(g)
len(g)
len(g.nodes())
len(g.edges())
# This matches the manual method above

1774016

networkx.classes.graph.Graph

389971

389971

1774016

<span style='background:purple;font-size:30px;color:white'>Proof that the THREE different edgelist formulas result in the exact same edges</span>


In [131]:
len(pd.DataFrame(dict2.keys()))
len(pd.DataFrame(edgeset))

#dfedges = pd.DataFrame(list(dict2.keys()),columns=['source','target'])
dict2edges = pd.Series(dict2).rename_axis(['source', 'target']).reset_index(name='weight')
dfedgeset = pd.DataFrame(edgeset, columns=['source','target'] )


1774016

1774016

In [132]:
sourcedtwo = dict2edges['source'].unique()
targetdtwo = dict2edges['target'].unique()
dtwonodes = np.unique(np.concatenate([sourcedtwo, targetdtwo]))
len(dtwonodes)
dtwolist = sorted(list(dtwonodes))

389971

In [133]:
sourcees = dfedgeset['source'].unique()
targetes = dfedgeset['target'].unique()
esnodes = np.unique(np.concatenate([sourcees, targetes]))
len(esnodes)
eslist = sorted(list(esnodes))

389971

In [134]:
gnodes = sorted(list(g.nodes))

In [135]:
dtwolist==eslist==gnodes

True

***

In [136]:
type(edgeset)
set(itertools.islice(edgeset, 10))

set

{('B000DZH4V0', 'B0196AO1IG'),
 ('B002EDVPU0', 'B00FAROBZS'),
 ('B00CP1QLSC', 'B01C6Y5SKY'),
 ('B00IL7BJNQ', 'B01CVG81UG'),
 ('B00KHD7NAA', 'B00T3T2N90'),
 ('B00ME5YAPK', 'B00UR321B6'),
 ('B00U3AW7EE', 'B00VHTHCHW'),
 ('B0145S680I', 'B07CQL2HHQ'),
 ('B01FREPEIQ', 'B01NCAL4DR'),
 ('B01G6ZJ6DY', 'B07BFH96M3')}

<span style='background:purple;font-size:30px;color:white'>Build Final Edgelist DataFrame</span>


In [137]:
#dfedges
len(dfedges)
dfedges.head()

#df_ratings = df_ratings.merge(dfnodes,how="left", left_on=['item'], right_on=['asin'])

1774016

,source,target,weight
0,B0047AFZXG,B0045EQCCW,2
1,B0047AFZXG,B00ZSHDJ4O,3
2,B0047AFZXG,B00MOUBYDQ,3
3,B0047AFZXG,B00NEMG62M,2
4,B0047AFZXG,B0033Z2BAQ,1


<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Create a Lookup Key that concatenates source+target asin strings</b></p>
</div>


In [138]:
dfedges['lookup'] =  dfedges['source'] + dfedges['target']
dfedges.sort_values(by='weight', ascending=False)

,source,target,weight,lookup
55398,B015PRO512,B017T2QB22,223,B015PRO512B017T2QB22
476607,B01FLK2MZ4,B01LYNQXCP,204,B01FLK2MZ4B01LYNQXCP
55431,B015PRO512,B0781Z7Y3S,152,B015PRO512B0781Z7Y3S
55435,B015PRO512,B00077INZU,143,B015PRO512B00077INZU
55403,B015PRO512,B00OJZUQ24,100,B015PRO512B00OJZUQ24
...,...,...,...,...
655906,B01GKK2AYW,B0143RTB1E,1,B01GKK2AYWB0143RTB1E
655905,B01GKK2AYW,B01ESSWXH2,1,B01GKK2AYWB01ESSWXH2
655903,B01GKK2AYW,B00CGTMXNK,1,B01GKK2AYWB00CGTMXNK
655902,B01GKKCZPG,B01638N2HK,1,B01GKKCZPGB01638N2HK


In [139]:
#df_weights = df_weights.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [140]:
## These needs to be an innerjoin followed by a left join with drop na, because if either source or node is missing metadata we need to drop

In [141]:
dfmerge1 = dfedges.merge(df_masterMETA.rename(columns={'title': 'target_title'}), how='inner', left_on=['target'], right_on=['asin'])


In [142]:
len(dfmerge1)
dfmerge1.head()

803381

,source,target,weight,lookup,target_title,asin
0,B0047AFZXG,B0045EQCCW,2,B0047AFZXGB0045EQCCW,iStar TC-RAIL-20 20-Inch Sliding Rail Kit for Most Rackmount Chassis,B0045EQCCW
1,B002MWDRD6,B0045EQCCW,1,B002MWDRD6B0045EQCCW,iStar TC-RAIL-20 20-Inch Sliding Rail Kit for Most Rackmount Chassis,B0045EQCCW
2,B00A7NBO6E,B0045EQCCW,2,B00A7NBO6EB0045EQCCW,iStar TC-RAIL-20 20-Inch Sliding Rail Kit for Most Rackmount Chassis,B0045EQCCW
3,B005KQ66ZC,B0045EQCCW,1,B005KQ66ZCB0045EQCCW,iStar TC-RAIL-20 20-Inch Sliding Rail Kit for Most Rackmount Chassis,B0045EQCCW
4,B01DI2BYPQ,B0045EQCCW,1,B01DI2BYPQB0045EQCCW,iStar TC-RAIL-20 20-Inch Sliding Rail Kit for Most Rackmount Chassis,B0045EQCCW


In [143]:
df_EDGES = dfmerge1.merge(df_masterMETA.rename(columns={'title': 'src_title'}), how='left', left_on=['source'], right_on=['asin']).dropna()[['source', 'target','weight', 'src_title', 'target_title','lookup']].copy()

In [144]:
len(df_EDGES)
df_EDGES.head()

843011

,source,target,weight,src_title,target_title,lookup
0,B0047AFZXG,B0045EQCCW,2,Effy Pav= Classica Collection Sterling Silver .97 ct. T.W. Diamond Bangle,iStar TC-RAIL-20 20-Inch Sliding Rail Kit for Most Rackmount Chassis,B0047AFZXGB0045EQCCW
1,B002MWDRD6,B0045EQCCW,1,StarTech.com 2.5in SATA Removable Hard Drive Bay for PC Expansion Slot,iStar TC-RAIL-20 20-Inch Sliding Rail Kit for Most Rackmount Chassis,B002MWDRD6B0045EQCCW
2,B00A7NBO6E,B0045EQCCW,2,iStarUSA Group D-214-MATX 2U 2 x 5.25 Compact Rackmount,iStar TC-RAIL-20 20-Inch Sliding Rail Kit for Most Rackmount Chassis,B00A7NBO6EB0045EQCCW
3,B005KQ66ZC,B0045EQCCW,1,iStar D Value D-416 4U Rackmount Server Chassis (Black),iStar TC-RAIL-20 20-Inch Sliding Rail Kit for Most Rackmount Chassis,B005KQ66ZCB0045EQCCW
4,B01DI2BYPQ,B0045EQCCW,1,EZDIY Front Panel for 3.5 inch Bay with Card Reader and Multi-Port Hub - USB 2.0/3.0,iStar TC-RAIL-20 20-Inch Sliding Rail Kit for Most Rackmount Chassis,B01DI2BYPQB0045EQCCW


In [145]:
df_EDGES[df_EDGES.weight.isnull()]

,source,target,weight,src_title,target_title,lookup


In [146]:
df_EDGES.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 843011 entries, 0 to 843013
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   source        843011 non-null  object
 1   target        843011 non-null  object
 2   weight        843011 non-null  int64 
 3   src_title     843011 non-null  string
 4   target_title  843011 non-null  string
 5   lookup        843011 non-null  object
dtypes: int64(1), object(3), string(2)
memory usage: 45.0+ MB


<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Check for duplicates via lookup</b></p>
</div>


In [148]:
df_EDGES[df_EDGES.duplicated(subset=['lookup'], keep=False)].sort_values(by='lookup', ascending=False).reset_index(drop=True)

,source,target,weight,src_title,target_title,lookup
0,B01HJ7XYFY,B0002J226O,1,"DB DRIVE WDX6.5 3K WDX 3K Series Competition Subwoofer (6.5&quot;, 600 Watts)",PAC LC-1 Remote Amplifier Level Controller,B01HJ7XYFYB0002J226O
1,B01HJ7XYFY,B0002J226O,1,"DB DRIVE WDX6.5 3K WDX 3K Series Competition Subwoofer (6.5&quot;, 600 Watts)",PAC LC-1 Remote Amplifier Level Controller,B01HJ7XYFYB0002J226O
2,B01HIZEW1C,B00017LSPI,1,AmazonBasics DSLR and Laptop Backpack,Giottos AA1900 Rocket Air Blaster Large - Black,B01HIZEW1CB00017LSPI
3,B01HIZEW1C,B00017LSPI,1,AmazonBasics DSLR and Laptop Backpack,Giottos AA1900 Rocket Air Blaster Large - Black,B01HIZEW1CB00017LSPI
4,B01HIZEW1C,B000053HH5,1,AmazonBasics DSLR and Laptop Backpack,Canon EF 70-200mm f/4L USM Telephoto Zoom Lens for Canon SLR Cameras,B01HIZEW1CB000053HH5
...,...,...,...,...,...,...
137339,1480338761,B0002D0COE,1,OEUVRES POUR GUITARE - GUITAR WORKS I - SOLO GUITAR,D'Addario Acoustic Guitar Humidifier,1480338761B0002D0COE
137340,1464120153,B00000JBNX,1,Iclicker+ Student Remote,"Texas Instruments TI-30X IIS 2-Line Scientific Calculator, Black with Blue Accents",1464120153B00000JBNX
137341,1464120153,B00000JBNX,1,Iclicker+ Student Remote,"Texas Instruments TI-30X IIS 2-Line Scientific Calculator, Black with Blue Accents",1464120153B00000JBNX
137342,1464120153,B00000JBNS,1,Iclicker+ Student Remote,Texas Instruments TI-30Xa Scientific Calculator,1464120153B00000JBNS


<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Eliminate Duplicate Edges</b></p>
</div>


In [149]:
df_EDGES = df_EDGES.drop_duplicates(subset=['lookup'], keep='last')

In [150]:
len(df_EDGES)

767299

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Export Final Cleaned up Edgeslist to CSV</b></p>
</div>


In [160]:
# compression_opts = dict(method='zip',
#                         archive_name='FINALELECTRONICS.csv')  
df_EDGES.to_csv('NOV23_ELECTRONICS.csv', index=False)  

In [151]:
df_EDGES

,source,target,weight,src_title,target_title,lookup
0,B0047AFZXG,B0045EQCCW,2,Effy Pav= Classica Collection Sterling Silver .97 ct. T.W. Diamond Bangle,iStar TC-RAIL-20 20-Inch Sliding Rail Kit for Most Rackmount Chassis,B0047AFZXGB0045EQCCW
1,B002MWDRD6,B0045EQCCW,1,StarTech.com 2.5in SATA Removable Hard Drive Bay for PC Expansion Slot,iStar TC-RAIL-20 20-Inch Sliding Rail Kit for Most Rackmount Chassis,B002MWDRD6B0045EQCCW
2,B00A7NBO6E,B0045EQCCW,2,iStarUSA Group D-214-MATX 2U 2 x 5.25 Compact Rackmount,iStar TC-RAIL-20 20-Inch Sliding Rail Kit for Most Rackmount Chassis,B00A7NBO6EB0045EQCCW
3,B005KQ66ZC,B0045EQCCW,1,iStar D Value D-416 4U Rackmount Server Chassis (Black),iStar TC-RAIL-20 20-Inch Sliding Rail Kit for Most Rackmount Chassis,B005KQ66ZCB0045EQCCW
4,B01DI2BYPQ,B0045EQCCW,1,EZDIY Front Panel for 3.5 inch Bay with Card Reader and Multi-Port Hub - USB 2.0/3.0,iStar TC-RAIL-20 20-Inch Sliding Rail Kit for Most Rackmount Chassis,B01DI2BYPQB0045EQCCW
...,...,...,...,...,...,...
843009,B007AJZMIO,B008F68SIM,1,Vortex Optics Pro GT Tripod Series,"Sportsman's Outdoor Products Snugfit Scope Skin Vortex Razor (85-mm, Angled)",B007AJZMIOB008F68SIM
843010,B007AJZMIO,B01BDBX4WU,1,Vortex Optics Pro GT Tripod Series,"SOG F11BN-CP Wood &amp; Bone Folding Saw, Steel Blades",B007AJZMIOB01BDBX4WU
843011,B007AJZMIO,B00ATNHPSI,1,Vortex Optics Pro GT Tripod Series,Outdoor Research Backcountry Organizer,B007AJZMIOB00ATNHPSI
843012,B007AJZMIO,B00CR1V8NS,1,Vortex Optics Pro GT Tripod Series,King's Camo Cotton Short Sleeve Hunting Tee,B007AJZMIOB00CR1V8NS


In [159]:
df_EDGES[df_EDGES.duplicated(subset=['src_title', 'source','target', 'target_title'])]

,source,target,weight,src_title,target_title,lookup
